In [ ]:
import torch

from src.pytorch_playground.utils import data_setup, model_builder, train



In [ ]:
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
                                                                train_dir=path/to/train_dir,
                                                                test_dir=path/to/test_dir,
                                                                transform=some_transform,
                                                                batch_size=32,
                                                                num_workers=4
                                                                )

In [ ]:
torch.manual_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
model = model_builder.TinyVGG(input_shape=3,
                              hidden_units=10, 
                              output_shape=len(class_names)).to(device)

In [ ]:
train.train(model=, 
            train_dataloader=, 
            test_dataloader=, 
            optimizer=,
            loss_fn=,
            epochs=,
            device=)